In [1]:
import requests 
import pandas as pd
import base64

#### Auth

In [ ]:
def auth_opc():
    SERVER_URL = 'primavera-us2.oraclecloud.com'
    api_URL=f'http://{SERVER_URL}/api'
    user = 'testuser'
    password = 'testpwd'
    url=f'https://{SERVER_URL}/primediscovery/apitoken/request?scope={api_URL}&user={user}&password={password}'

    usrPass = f"{user}:{password}"
    b64Val = base64.b64encode(usrPass.encode('ascii'))

    headers = {"Authorization": f"Basic {b64Val.decode('ascii')}",
               "Content-type": 'application/json'}
    
    r=requests.post(url, headers=headers, data={})

    init_response = r.json()
    accessToken = init_response['accessToken']
    auth_tokens = init_response['requestHeaders']

    auth_tokens["Authorization"] = f"Bearer {accessToken}"
    auth_tokens["Content-Type"] = "application/json"
    return auth_tokens

#### Extract MetaData

In [ ]:
def get_tables(meta_url, header):
    tables_path = '/data/rest/dataservice/tables'
    url = meta_url+tables_path
    r=requests.get(url, headers=header, data={})
    tbl_df = pd.json_normalize(r.json())
    return tbl_df

def get_columns(meta_url, header, table_name):
    cols_path = f'/data/rest/dataservice/columns/{table_name}'
    url = meta_url+cols_path
    r=requests.get(url, headers=header, data={})
    cols_df = pd.json_normalize(r.json())
    cols = cols_df.columnName.to_list()
    return cols

#### Pagination

In [ ]:
# Get Table Data 
def get_data(table_name, cols, headers,nkey,ntablename):
    data_url = 'https://primavera-us2.oraclecloud.com'
    run_qry = '/data/rest/dataservice/runquery'

    query = {
        "name":f"Get Data for {table_name}",
        "nextKey": nkey,
        "nextTableName": ntablename,
        "tables":[{
        "tableName": table_name,
        "columns": cols
        }]
    }

    res_data = requests.post(data_url+run_qry, headers=headers, json=query)
 
    data_df,nextkey,nexttable = pd.json_normalize(res_data.json()['data'][table_name]),res_data.json()['data']['pagination'][0]['nextKey'],res_data.json()['data']['pagination'][0]['nextTableName']

    return data_df,nextkey,nexttable

#### Load Data for all Tables

In [ ]:
headers = auth_opc()
meta_url = 'http://primavera-us2.oraclecloud.com'

tables_df = get_tables(meta_url, headers)
tables_df.head()

In [ ]:
for idx, row in tables_df.iterrows():
    print(row.physicalTableName)
    df = pd.DataFrame()
    table_name = row.physicalTableName
    cols = get_columns(meta_url, headers, table_name)
    nkey=-999
    while int(nkey)==-999 or (int(nkey) > 0 and int(nkey) != "") :
        if int(nkey)==-999:
            data_df,nkey,ntable = get_data(table_name, cols, headers,"","")
            data_final=data_df
        else:
            data_df,nkey,ntable = get_data(table_name, cols, headers,nkey,table_name)
            data_final = pd.concat([data_final, data_df], ignore_index=True)
            
    data_final.to_csv(f'data\{table_name}.csv',index=False)
    if idx > 1:
        break